# Hướng Dẫn Sử Dụng

Faceswap - Google Colab

Bạn cũng cần Faceswap GUI để làm việc này, vì bạn chưa thể trích xuất khuôn mặt ở đây.

Các tệp bạn cần tải lên Google Drive:
1.   face_a.zip (ảnh khuôn mặt nguồn mà bạn muốn hoán đổi)
2.   face_b.zip (khuôn mặt mà bạn muốn được hoán đổi)

> Các tệp nén với khuôn mặt đã trích xuất và tệp alignments.fsa

3.   train.ini
> Tệp train.ini từ hệ thống local của bạn, trên Windows nằm ở: C:\Users\TênBạn\faceswap\config

Cấu trúc thư mục trên Google Drive:
*   /colab_files/faceswap/faces/face_a.zip
*   /colab_files/faceswap/faces/face_b.zip
*   /colab_files/faceswap/config/train.ini

Các thư mục khuyến nghị tạo trên Google Drive:
*   /colab_files/faceswap/models/TênModelCủaBạn
*   /colab_files/faceswap/output/timelapse

Cấu trúc thư mục trong Google Colab sau bước "Gắn kết Google Drive" bên dưới:
*   /content/drive/My Drive/colab_files/faceswap/faces/face_a.zip
*   /content/drive/My Drive/colab_files/faceswap/faces/face_b.zip
*   /content/drive/My Drive/colab_files/faceswap/config/train.ini

# Ghi Chú Sử Dụng

Đây là quy trình chuẩn (SOP) cá nhân của tôi để xử lý quá trình Faceswap. Tôi thấy tốt nhất là điều chỉnh cài đặt, trích xuất, kiểm tra huấn luyện và chuyển đổi mọi thứ bằng hệ thống local của bạn.

Nhưng việc huấn luyện trên laptop của tôi mất vài tuần để có được kết quả tương tự mà bạn có thể đạt được chỉ trong vài ngày bằng cách sử dụng Google Colab.

*   Trích xuất (Extract), Căn chỉnh (Align), sau đó nén (Zip) tất cả Faces A
*   Trích xuất (Extract), Căn chỉnh (Align), sau đó nén (Zip) tất cả Faces B
*   Chỉnh sửa script "Chạy Huấn Luyện" bên dưới theo sở thích của bạn
*   Huấn luyện trong Colab Notebook này
*   Theo dõi thư mục timestamp trên Google Drive để xem quá trình huấn luyện đang cải thiện như thế nào
*   Tải xuống TênModelCủaBạn về hệ thống local
*   Chuyển đổi (Convert) bằng model đã tải xuống

# Giữ Phiên Hoạt Động
 (có thể không hoạt động do thay đổi Captcha mới nhất của Google Colab.)

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# Kiểm Tra GPU Của Bạn
Nếu không phải **Tesla T4** hoặc **Tesla P100**, hãy vào Runtime->Factory Reset Runtime cho đến khi bạn nhận được một trong hai loại này.

In [ ]:
!nvidia-smi

import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

# Cài Đặt Faceswap

In [ ]:
#@title Set Time Zone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Turkey /etc/localtime
!date

#above is for HST, you can find yours in
#/usr/share/zoneinfo


Sat May 29 14:19:09 +03 2021


In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#@title Get your training data(aka the faces you extracted in app)
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip" .

!unzip face_a.zip -d face_a
!unzip face_b.zip -d face_b

!rm face_a.zip
!rm face_b.zip


In [ ]:
#@title Get the latest Faceswap
!git clone https://github.com/deepfakes/faceswap.git

In [ ]:
#@title Copy your configuration(train.ini) file
!cp "/content/drive/My Drive/colab_files/faceswap/config/train.ini" faceswap/config/
!ls -lA faceswap/config/
!cat faceswap/config/train.ini

In [ ]:
#@title Install Tensorflow

!pip install -r faceswap/requirements_nvidia.txt

In [ ]:
!python3 faceswap/tools.py mask -a 'face_b/alignments.fsa' -i 'face_b' -it faces -M vgg-clear -p all -o '/drive/MyDrive/colab_files/faceswap/faces/facexb/mask' -b 3 -t 4 -ot mask -L INFO


# Chạy Huấn Luyện
Số lượng vòng lặp (iteration) tùy thuộc vào bạn nhưng khuyến nghị bạn nên thực hiện cho đến khi đạt được số lượng mà bạn hài lòng. Bạn nên kiểm tra thư mục timelapse của mình để theo dõi.

Và nếu tính toán của tôi đúng, trainer đang thực hiện gần 360 vòng lặp trong 10 phút, tức là 2160 vòng lặp trong 1 giờ.

In [ ]:
#set variables start
num_iterations = "100000"
save_every = "360"
save_model_every = "25000"
batch_num = "16"
num_gpus = "1"

trainer_type = "villain"

model_dir = "/content/drive/My Drive/colab_files/faceswap/models/YourModelName"
alignments_file_a = "face_a/2_alignments.fsa"
alignments_file_b = "face_b/alignments.fsa"
timelapse_dir = "/content/drive/My Drive/colab_files/faceswap/output/timelapse"
#set variables end

!python3 faceswap/faceswap.py train \
  -A 'face_a' \
  -B 'face_b'  \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia 'face_a' \
  -tib 'face_b' \
  -to '{timelapse_dir}'


# Chuyển Đổi Video

In [ ]:
!python3 faceswap/faceswap.py convert -i '/content/drive/My Drive/colab_files/faceswap/Editor.avi' -o '/content/drive/My Drive/colab_files/faceswap/Results' -al '/content/drive/My Drive/colab_files/faceswap/2_alignments.fsa' -m '/content/drive/My Drive/colab_files/faceswap/models/YourModelName' -c match-hist -M none -w ffmpeg -osc 100 -l 0.4 -j 0 -t realface -L INFO